# Processing cyclic voltammograms from one file set

## LIbrary imports and some default values for plotting

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
from ipywidgets import widgets
from ipywidgets import interact, interactive, Button, HBox, VBox, Text, interactive_output

In [ ]:
ref_electrodes = ['Li/Li$^+$', 'Na/Na$^+$', 'Hg/HgSO$_4$', 'Ag/AgCl', 'Carbon', 'Ag']
normalization_options = ['Current density (A/g)', 'Specific Capacitance (F/g)', 'Specific Capacity (mAh/g)', 
                         'Specific Charge (C/g)', 'Volumetric Capacitance (F/cm$^3$)', 'Volumetric Capacity (mAh/cm$^3$)',
                        'Volumetric Charge (C/cm$^3$)', 'Areal Capacitance (F/cm$^2$)', 'Areal Capacity (mAh/cm$^2$)',
                        'Areal Charge (C/cm$^2$)']

## Reading in files and input of user constants

In [ ]:
# Enter path to folder containing your data in quotes followed by /*. Should match "your_path/*"
file_list = glob.glob('/*')
file_list.sort()

In [ ]:
file_list

In [ ]:
# Enter your scan rates as a comma separated list in the square brackets, e.g., [5, 10, 20, 50, 100]
scan_rates = []

In [ ]:
# Replace 'None' with your values. Okay to leave as 'None' if you don't have a value.
electrode_mass = None #in mg
electrode_area = None #in cm^2
electrode_volume = None #in cm^3

## BioLogic cyclic voltammograms

In [ ]:
def cv_avg_and_error(data, scan_rate):
    '''Calculates average potential, current (and corresponding error) 
    for the second to n - 1 cycles for a single scan rate.'''
    
    cycle_num = data['cycle number'].unique()
    
    potential = []
    for n in cycle_num[1:-1]:
        potential.append(np.asarray(data['Ewe/V'][data['cycle number'] == n]))
    potential = pd.DataFrame(potential).transpose()
    avg_potential = potential.mean(axis=1)
    
    current = []
    for n in cycle_num[1:-1]:
        current.append(np.asarray(data['<I>/mA'][data['cycle number'] == n]))
    current = pd.DataFrame(current).transpose()
    avg_current = np.mean(current, axis=1)
    current_err = np.std(current, axis=1)

    return avg_potential, avg_current, current_err

In [ ]:
def plot_CVs_biologic(ref, yaxis, Error):
    '''Plots average voltammograms from file list with error bands.'''
    global fig
    
    fig, ax = plt.subplots(figsize=(6, 6), dpi=150)

    for file, rate in zip(file_list, scan_rates):
        
        with open(file, 'r') as f:
            data = pd.read_table(f)
            avg_potential, avg_current, current_err = cv_avg_and_error(data, rate)
            
            if 'Capacity' in yaxis:
                capacity = (avg_current * (max(avg_potential) - min(avg_potential))) / (3.6 * rate)
                capacity_err = (current_err * (max(avg_potential) - min(avg_potential))) / (3.6 * rate)
                
                if 'Specific' in yaxis:
                    ax.plot(avg_potential, 
                            capacity * 1000 / electrode_mass, 
                            linewidth=2, label=(f"{rate} mV/s"))
                    if Error:
                        ax.fill_between(avg_potential, 
                                        (capacity + capacity_err) * 1000 / electrode_mass, 
                                        (capacity - capacity_err) * 1000 / electrode_mass, 
                                        alpha=0.4)
                elif 'Volumetric' in yaxis:
                    ax.plot(avg_potential, 
                            capacity / electrode_volume, 
                            linewidth=2, label=(f"{rate} mV/s"))
                    if Error:
                        ax.fill_between(avg_potential, 
                                        (capacity + capacity_err) / electrode_volume, 
                                        (capacity - capacity_err) / electrode_volume, 
                                        alpha=0.4)
                elif 'Areal' in yaxis:
                    ax.plot(avg_potential, 
                            capacity / electrode_area, 
                            linewidth=2, label=(f"{rate} mV/s"))
                    if Error:  
                        ax.fill_between(avg_potential, 
                                        (capacity + capacity_err) / electrode_area, 
                                        (capacity - capacity_err) / electrode_area, 
                                        alpha=0.4)
                    
            elif 'Capacitance' in yaxis:
                capacitance = avg_current / rate
                capacitance_err = current_err / rate
                
                if 'Specific' in yaxis:
                    ax.plot(avg_potential, 
                            capacitance * 1000 / electrode_mass, 
                            linewidth=2, label=(f"{rate} mV/s"))
                    if Error:
                        ax.fill_between(avg_potential, 
                                        (capacitance + capacitance_err) * 1000 / electrode_mass, 
                                        (capacitance - capacitance_err) * 1000 / electrode_mass, 
                                        alpha=0.4)
                elif 'Volumetric' in yaxis:
                    ax.plot(avg_potential, 
                            capacitance / electrode_volume, 
                            linewidth=2, label=(f"{rate} mV/s"))
                    if Error:
                        ax.fill_between(avg_potential, 
                                        (capacitance + capacitance_err) / electrode_volume, 
                                        (capacitance - capacitance_err) / electrode_volume, 
                                        alpha=0.4)
                elif 'Areal' in yaxis:
                    ax.plot(avg_potential, 
                            capacitance / electrode_area, 
                            linewidth=2, label=(f"{rate} mV/s"))
                    if Error:  
                        ax.fill_between(avg_potential, 
                                        (capacitance + capacitance_err) / electrode_area, 
                                        (capacitance - capacitance_err) / electrode_area, 
                                        alpha=0.4)
            
            elif 'Charge' in yaxis:
                charge = (avg_current * (max(avg_potential) - min(avg_potential))) / rate
                charge_err = (current_err * (max(avg_potential) - min(avg_potential))) / rate
                
                if 'Specific' in yaxis:
                    ax.plot(avg_potential, charge * 1000 / electrode_mass, 
                            linewidth=2, label=(f"{rate} mV/s"))
                    if Error:
                        ax.fill_between(avg_potential, 
                                        (charge + charge_err) * 1000 / electrode_mass, 
                                        (charge - charge_err)* 1000 / electrode_mass, 
                                        alpha=0.4)
                elif 'Volumetric' in yaxis:
                    ax.plot(avg_potential, charge / electrode_volume, 
                            linewidth=2, label=(f"{rate} mV/s"))
                    if Error:
                        ax.fill_between(avg_potential, 
                                        (charge + charge_err) / electrode_volume, 
                                        (charge - charge_err) / electrode_volume, 
                                        alpha=0.4)
                elif 'Areal' in yaxis:
                    ax.plot(avg_potential, charge / electrode_area, 
                            linewidth=2, label=(f"{rate} mV/s"))
                    if Error:  
                        ax.fill_between(avg_potential, 
                                        (charge + charge_err) / (electrode_area), 
                                        (charge - charge_err) / (electrode_area), 
                                        alpha=0.4)
            
            else:
                ax.plot(avg_potential, avg_current / electrode_mass, linewidth=2, label=(f"{rate} mV/s"))
                if Error:
                    ax.fill_between(avg_potential, (avg_current + current_err) / electrode_mass, 
                                    (avg_current - current_err) / electrode_mass, 
                                    alpha=0.4)
                          
    
    ax.tick_params(which='both', labelsize=14, width=2, length=5)
    ax.legend(frameon=False)

    ax.set_xlabel(f'Potential vs. {ref}', fontsize=18)
    ax.set_ylabel(f'{yaxis}', fontsize=18)

In [ ]:
def make_figure():
    refs = widgets.Dropdown(options=ref_electrodes)
    yaxis_choice = widgets.Dropdown(options=normalization_options)
    err = widgets.Checkbox(description='Error', indent=False)
    def on_button_clicked(b):
        fig.savefig("test.png")

    button = Button(description="Save Figure")
    button.on_click(on_button_clicked)

    ui = HBox([refs, yaxis_choice, err, button])

    out = widgets.interactive_output(plot_CVs_biologic, {'ref': refs, 'yaxis': yaxis_choice, 'Error': err})
    display(ui, out)

In [ ]:
make_figure()